# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [ ]:
%load_ext dotenv
%dotenv ../01_materials/labs/.env


In [3]:
# Turn the query planning option on to prevent message
import dask
dask.config.set({'dataframe.query-planning': True})

import dask.dataframe as dd

c:\Users\Anna\miniconda3\envs\dsi_participant\lib\site-packages\dask\dataframe\_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(


+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [ ]:
import os
from glob import glob

#Retrieve the value of the PRICE_DATA
PRICE_DATA = os.getenv('PRICE_DATA')
parquet_files = glob(os.path.join(PRICE_DATA, "**/*parquet/part.0.parquet"))

# Read a parquet files into a Dask dataframe and sets the 'ticker'
dd_p = dd.read_parquet(parquet_files).set_index("ticker")


In [5]:
dd_p.head()

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year
ticker,,,,,,,,,,
A,2000-01-03,56.330471,56.464592,48.193848,51.502148,43.463028,4674353,Health Care,Life Sciences Tools & Services,2000
A,2000-01-04,48.730328,49.266811,46.316166,47.567955,40.142933,4765083,Health Care,Life Sciences Tools & Services,2000
A,2000-01-05,47.389126,47.567955,43.141991,44.617310,37.652866,5758642,Health Care,Life Sciences Tools & Services,2000
A,2000-01-06,44.080830,44.349072,41.577251,42.918453,36.219189,2534434,Health Care,Life Sciences Tools & Services,2000
A,2000-01-07,42.247852,47.165592,42.203148,46.494991,39.237453,2819626,Health Care,Life Sciences Tools & Services,2000


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [ ]:
import numpy as np
import pandas as pd

#Define columns type
column_types = {
    'Date': 'datetime64[ns, UTC]',
    'Open': float,
    'High': float,
    'Low': float,
    'Close': float,
    'Adj Close': float, 
    'Volume': np.int64,
    'sector': 'string[pyarrow]',
    'subsector': 'string[pyarrow]',
    'year': 'int32',
    'Close_lag': float,
    'Adj_Close_lag': float,
}

#Define metadata for Dask
meta_df = pd.DataFrame({col: pd.Series(dtype=dt) for col, dt in column_types.items()})

#Add features using chain of apply(), lambda, and assign()
dd_feat = (
    dd_p.groupby('ticker', group_keys=False).apply(
        lambda x: x.assign(
            # Add lags for 'Close' and 'Adj_Close'
                Close_lag = x['Close'].shift(1),
                Adj_Close_lag = x['Adj Close'].shift(1),
            )
            , meta = meta_df
        )
        .assign(
            # Calculate the daily high-low range
            hi_lo_range = lambda x: x['High'] - x['Low']
        )
        .assign(
            # Calculate returns based on Adjusted Close
            returns = lambda x: x['Adj Close'] / x['Adj_Close_lag'] - 1
        )
)

+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [ ]:
#Save the dataframe as a Pandas dataframe
pd_feat = dd_feat.compute()


In [9]:
#Calculate the 10-day rolling average return
pd_feat['avg_return_10d'] = pd_feat.groupby('ticker')['returns'].transform(lambda s: s.rolling(10).mean())
pd_feat

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Close_lag,Adj_Close_lag,hi_lo_range,returns,avg_return_10d
ticker,,,,,,,,,,,,,,,
HUM,2014-01-02,102.760002,103.879997,102.300003,102.839996,95.121140,1393500,Health Care,Managed Health Care,2014,NaN,NaN,1.579994,NaN,NaN
HUM,2014-01-03,102.860001,103.040001,101.580002,101.760002,94.122192,1234400,Health Care,Managed Health Care,2014,102.839996,95.121140,1.459999,-0.010502,NaN
HUM,2014-01-06,102.300003,102.379997,99.959999,100.760002,93.197258,1850200,Health Care,Managed Health Care,2014,101.760002,94.122192,2.419998,-0.009827,NaN
HUM,2014-01-07,98.720001,101.349998,98.500000,100.550003,93.003036,2561700,Health Care,Managed Health Care,2014,100.760002,93.197258,2.849998,-0.002084,NaN
HUM,2014-01-08,100.660004,100.660004,98.949997,99.400002,91.939339,2332700,Health Care,Managed Health Care,2014,100.550003,93.003036,1.710007,-0.011437,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
JNPR,2018-12-24,26.110001,26.320000,25.559999,25.559999,21.583513,2061500,Information Technology,Communications Equipment,2018,26.230000,22.149281,0.760000,-0.025543,-0.007540
JNPR,2018-12-26,25.660000,26.600000,25.299999,26.600000,22.461720,4847500,Information Technology,Communications Equipment,2018,25.559999,21.583513,1.300001,0.040689,-0.004268
JNPR,2018-12-27,26.219999,26.799999,25.840000,26.799999,22.630602,2878300,Information Technology,Communications Equipment,2018,26.600000,22.461720,0.959999,0.007519,-0.005779


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

Answer: It wasn't necessary to convert the data to Pandas DataFrame to calculate the moving average return. Dask has .rolling() function similar to Pandas, so we could perform the operation directly in Dask. This would often be more efficient, especially with large datasets, because Dask uses parallel processing, which makes it faster than Pandas for such tasks. 

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.